In [1]:
# !pip install evidently

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip list

Package               Version
--------------------- -----------
altair                4.2.0
argon2-cffi           21.3.0
argon2-cffi-bindings  21.2.0
asttokens             2.0.8
attrs                 22.1.0
autoviz               0.1.43
backcall              0.2.0
beautifulsoup4        4.11.1
bleach                5.0.1
bokeh                 2.4.3
certifi               2022.6.15
cffi                  1.15.1
charset-normalizer    2.1.1
click                 8.1.3
colorama              0.4.5
colorcet              3.0.0
cycler                0.11.0
dataclasses           0.6
debugpy               1.6.3
decorator             5.1.1
defusedxml            0.7.1
emoji                 2.0.0
entrypoints           0.4
evidently             0.1.56.dev0
executing             0.10.0
fastjsonschema        2.16.1
fonttools             4.37.0
fsspec                2022.7.1
greenlet              1.1.3
holoviews             1.14.9
htmlmin               0.1.12
hvplot                0.8.0
idna               


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Libraries

In [4]:
# import sys
# print(sys.path)
# sys.path.insert(0, "C:\\Users\\KonuTech\\gfk_data_analyst\\gfk_ml_ops_data_analyst\\scripts\\python\\")

In [5]:
import os
# import sys
import json
import numpy as np
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
# import math
from pandas_profiling import ProfileReport
# import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scripts.python.get_metrics import *
from scripts.python.get_charts import get_monthly_stability_chart
from scripts.python.get_reports import *

In [6]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [7]:
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab, CatTargetDriftTab

In [8]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import ClassificationPerformanceTab

In [9]:
INPUT_PATH="input"
OUTPUT_PATH="output"
CONFIG_PATH="config"

INPUT_DATA="test_data2"
INPUT_DATA_CONFIG="input_config"
OUTPUT_DATA="output"

INPUT_EXTENSION="csv"
INPUT_CONFIG_EXTENSION="json"
OUTPUT_EXTENSION="csv"

INPUT_FILE=f"{INPUT_DATA}.{INPUT_EXTENSION}"
INPUT_CONFIG_FILE=f"{INPUT_DATA_CONFIG}.{INPUT_CONFIG_EXTENSION}"
OUTPUT_FILE=f"{OUTPUT_DATA}.{OUTPUT_EXTENSION}"

INPUT_ABS_APTH=os.path.abspath(os.path.join(INPUT_PATH, INPUT_FILE))
INPUT_FILE_CONFIG=os.path.abspath(os.path.join(CONFIG_PATH, INPUT_CONFIG_FILE))
OUTPUT_ABS_APTH=os.path.abspath(os.path.join(OUTPUT_PATH, OUTPUT_FILE))

In [10]:
with open(INPUT_FILE_CONFIG, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [11]:
CONFIG

{'INPUTS': {'FILE_NAME': ['test_data2.csv'],
  'SEPARATOR': ',',
  'DECIMAL': None,
  'ENCODING': 'utf-8',
  'FLOAT_PRECISION': 'high',
  'INDEXES': ['period_end_date', 'translated_when'],
  'DATE_COLUMNS': ['period_end_date', 'translated_when'],
  'DTYPE': {'if_data_corrected': 'object',
   'prod_gr_id': 'object',
   'country_id_n': 'object',
   'delivery_type_id': 'object',
   'freq_id': 'object',
   'retailer_id': 'object',
   'brand_id': 'object',
   'predict_automatch': 'float',
   'class_acctual': 'float'},
  'COLUMNS_WITH_NAN_VALUES': [None]},
 'MODEL': [None],
 'OUTPUTS': {'NAME': ['output.csv']}}

# Load data

In [12]:
# LOAD CSV
df = pd.read_csv(
    INPUT_ABS_APTH,
#     index_col=CONFIG['INPUTS']['INDEXES'],
    sep=CONFIG['INPUTS']['SEPARATOR'],
    # quotechar='"',
    # thousands=',',
#     decimal=CONFIG['INPUTS']['DECIMAL'],
    encoding=CONFIG['INPUTS']['ENCODING'],
    infer_datetime_format=True,
    parse_dates=CONFIG['INPUTS']['DATE_COLUMNS'],
    engine="c",
    low_memory=False,
#     float_precision=CONFIG['INPUTS']['DATE_COLUMNS'],
    skipinitialspace=True,
    dtype=CONFIG['INPUTS']['DTYPE']
)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19697 entries, 0 to 19696
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   period_end_date    19640 non-null  datetime64[ns, UTC]
 1   translated_when    19697 non-null  datetime64[ns, UTC]
 2   if_data_corrected  19697 non-null  object             
 3   prod_gr_id         19697 non-null  object             
 4   country_id_n       18405 non-null  object             
 5   delivery_type_id   18362 non-null  object             
 6   freq_id            19697 non-null  object             
 7   retailer_id        19697 non-null  object             
 8   brand_id           19697 non-null  object             
 9   predict_automatch  19368 non-null  float64            
 10  class_acctual      19697 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(2), object(7)
memory usage: 1.7+ MB


In [14]:
df.sort_values(
    by=['translated_when'],
    inplace=True
)

df.reset_index(
    drop=True,
    inplace=True
)

In [15]:
df.head(10)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
0,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,0,426,126,22866,1,174,280,1.0,1.0
1,2020-08-30 00:00:00+00:00,2020-09-01 03:35:26+00:00,0,426,126,22866,2,93,96,1.0,1.0
2,2020-08-30 00:00:00+00:00,2020-09-01 03:44:29+00:00,0,413,113,12731,2,95,182,1.0,1.0
3,2020-09-01 00:00:00+00:00,2020-09-01 06:15:32+00:00,1,426,114,18614,1,173,279,1.0,1.0
4,2020-09-01 00:00:00+00:00,2020-09-01 06:15:50+00:00,1,426,114,18614,1,34,234,1.0,1.0
5,2020-09-01 00:00:00+00:00,2020-09-01 06:21:31+00:00,1,426,114,18614,2,93,96,1.0,1.0
6,2020-09-01 00:00:00+00:00,2020-09-01 06:24:04+00:00,1,426,114,18614,1,117,243,1.0,1.0
7,2020-09-01 00:00:00+00:00,2020-09-01 06:24:13+00:00,1,426,114,18614,2,11,33,1.0,1.0
8,2020-09-01 00:00:00+00:00,2020-09-01 06:25:53+00:00,1,426,114,18614,1,26,169,0.0,0.0
9,2020-09-01 00:00:00+00:00,2020-09-01 06:26:45+00:00,1,426,114,18614,1,33,379,0.0,0.0


In [16]:
df.describe(
    include='all',
    datetime_is_numeric=True
)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
count,19640,19697,19697,19697,18405,18362,19697,19697,19697,19368.000000,19697.000000
unique,NaN,NaN,2,3,34,914,2,52,199,NaN,NaN
top,NaN,NaN,0,426,121,31480,2,30,33,NaN,NaN
freq,NaN,NaN,17085,11844,4153,998,11934,1197,587,NaN,NaN
mean,2020-10-14 23:30:22.729124096+00:00,2020-10-22 06:01:21.525206784+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.636307,0.743210
min,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,2020-09-27 00:00:00+00:00,2020-09-29 13:45:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,2020-10-18 00:00:00+00:00,2020-10-22 04:30:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
75%,2020-11-08 00:00:00+00:00,2020-11-13 11:57:53+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
max,2020-12-01 00:00:00+00:00,2021-02-01 14:50:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19697 entries, 0 to 19696
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   period_end_date    19640 non-null  datetime64[ns, UTC]
 1   translated_when    19697 non-null  datetime64[ns, UTC]
 2   if_data_corrected  19697 non-null  object             
 3   prod_gr_id         19697 non-null  object             
 4   country_id_n       18405 non-null  object             
 5   delivery_type_id   18362 non-null  object             
 6   freq_id            19697 non-null  object             
 7   retailer_id        19697 non-null  object             
 8   brand_id           19697 non-null  object             
 9   predict_automatch  19368 non-null  float64            
 10  class_acctual      19697 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(2), object(7)
memory usage: 1.7+ MB


In [18]:
df.head(3)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
0,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,0,426,126,22866,1,174,280,1.0,1.0
1,2020-08-30 00:00:00+00:00,2020-09-01 03:35:26+00:00,0,426,126,22866,2,93,96,1.0,1.0
2,2020-08-30 00:00:00+00:00,2020-09-01 03:44:29+00:00,0,413,113,12731,2,95,182,1.0,1.0


### Peak of events: ~Black Friday 2020, Friday 27th of November -- week 48

### TARGET DRIFT - global report

In [19]:
get_target_drift_report(
    data_frame=df,
    target='class_acctual',
    prediction='predict_automatch',
    datetime='translated_when',
    categorical_fatures=['country_id_n', 'prod_gr_id', 'retailer_id', 'brand_id', 'delivery_type_id', 'week_number'],
    columns_to_exclude=['period_end_date', 'if_data_corrected', 'freq_id'],
    breaking_point_dt='2020-11-28 00:00:00+00:00'
)

PRODUCED A CHART OF TARGET DRIFT GLOBAL


### TARGET DRIFT - weekly report

In [20]:
get_target_drift_report_weekly(
    data_frame=df,
    target='class_acctual',
    prediction='predict_automatch',
    datetime='translated_when',
    categorical_fatures=['country_id_n', 'prod_gr_id', 'retailer_id', 'brand_id', 'delivery_type_id', 'week_number'],
    columns_to_exclude=['period_end_date', 'if_data_corrected', 'freq_id']
)

PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202038
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202039
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202040
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202041
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202042
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202043
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202044
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202045
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202046
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202047
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202048
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202049
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202050
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202051
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202052
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202053
PRODUCED A CHART OF TARGET DRIFT WEEKLY FOR WEEK: 202101
PRODUCED A CHART OF TARGET DRIF

### DATA DRIFT - global report

In [21]:
get_data_drift_report(
    data_frame=df,
    target='class_acctual',
    prediction='predict_automatch',
    datetime='translated_when',
    categorical_fatures=['country_id_n', 'prod_gr_id', 'retailer_id', 'brand_id', 'delivery_type_id', 'week_number'],
    columns_to_exclude=['period_end_date', 'if_data_corrected', 'freq_id'],
    breaking_point_dt='2020-11-28 00:00:00+00:00'
)

PRODUCED A CHART OF DATA DRIFT GLOBAL


### DATA DRIFT - weekly report

In [22]:
get_data_drift_report_weekly(
    data_frame=df,
    target='class_acctual',
    prediction='predict_automatch',
    datetime='translated_when',
    categorical_fatures=['country_id_n', 'prod_gr_id', 'retailer_id', 'brand_id', 'delivery_type_id', 'week_number'],
    columns_to_exclude=['period_end_date', 'if_data_corrected', 'freq_id']
)

PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202038
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202039
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202040
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202041
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202042
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202043
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202044
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202045
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202046
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202047
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202048
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202049
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202050
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202051
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202052
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202053
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202101
PRODUCED A CHART OF DATA DRIFT WEEKLY FOR WEEK: 202102
PRODUCED A

### CLASSIFICATION PERFORMANCE - global report

In [23]:
get_classification_performance_report(
    data_frame=df,
    target='class_acctual',
    prediction='predict_automatch',
    datetime='translated_when',
    categorical_fatures=['country_id_n', 'prod_gr_id', 'retailer_id', 'brand_id', 'delivery_type_id', 'week_number'],
    columns_to_exclude=['period_end_date', 'if_data_corrected', 'freq_id'],
    breaking_point_dt='2020-11-28 00:00:00+00:00'
)

PRODUCED A CHART OF CLASSIFICATION PERFORMANCE GLOBAL


### CLASSIFICATION PERFORMANCE - weekly report

In [24]:
get_classification_performance_report_weekly(
    data_frame=df,
    target='class_acctual',
    prediction='predict_automatch',
    datetime='translated_when',
    categorical_fatures=['country_id_n', 'prod_gr_id', 'retailer_id', 'brand_id', 'delivery_type_id', 'week_number'],
    columns_to_exclude=['period_end_date', 'if_data_corrected', 'freq_id']
)

PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202038
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202039
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202040
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202041
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202042
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202043
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202044
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202045
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202046
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202047
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202048
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202049
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202050
PRODUCED A CHART OF LASSIFICATION PERFORMANCE WEEKLY FOR WEEK: 202051
PRODUCED A CHART OF 